In [1]:
import pandas as pd 

In [ ]:
dataset=pd.read_csv("https://raw.githubusercontent.com/Jomuel-net/AI-finance-project/refs/heads/main/Data%20set/data_bank_alaska.csv")
#bite

C:\Users\USER\AppData\Local\Temp\ipykernel_12444\918415353.py:1: DtypeWarning: Columns (36,38,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset=pd.read_csv("https://raw.githubusercontent.com/Jomuel-net/AI-finance-project/refs/heads/main/Data%20set/data_bank_alaska.csv")
